Шаг 0: Импорт библиотек
Для решения этого задания мы можем использовать библиотеки pandas, librosa для обработки аудио и torch или tensorflow для построения модели.

In [50]:
import os
import json
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Шаг 1: Подготовка данных
Мы начнем с того, что организуем данные, которые мы имеем, и подготовим их к обучению.

In [41]:
# Константы
DATA_DIR = '../data/train/'
DATA_DIR_FILES = [
    'hr_bot_clear',
    'hr_bot_noise',
    'hr_bot_synt'
]
ANNOTATION_DIR = '../data/train/annotation/'
ANNOTATION_FILES = [
    'hr_bot_clear.json',
    'hr_bot_noise.json',
    'hr_bot_synt.json'
]

# Загрузка аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Загрузка данных и их обработка
def load_dataset(data_dir, annotation_files):
    dataset = []
    # Перебираем файлы аннотаций
    for annotation_file in annotation_files:
        annotations = load_annotations(os.path.join(ANNOTATION_DIR, annotation_file))

        # Получаем соответствующую директорию на основе названия файла аннотации
        sub_dir = annotation_file.replace('.json', '')
        print(f"Loading from subdirectory: {sub_dir}")

        for item in annotations:
            if all(k in item for k in ['audio_filepath', 'text', 'label', 'attribute']):
                # Используем os.path.join для правильного формирования пути
                audio_path = os.path.join(data_dir, sub_dir, item['audio_filepath'])
                print(f"Checking audio file path: {audio_path}")

                if os.path.exists(audio_path):
                    dataset.append({
                        'audio_filepath': audio_path,
                        'text': item['text'],
                        'label': item['label'],
                        'attribute': item['attribute']
                    })
                else:
                    print(f"Audio file does not exist: {audio_path}")
            else:
                print(f"Missing keys in item: {item}")
    return dataset

# Загрузка аннотаций и создание DataFrame
dataset = load_dataset(DATA_DIR, ANNOTATION_FILES)
df = pd.DataFrame(dataset)

Loading from subdirectory: hr_bot_clear
Checking audio file path: ../data/train/hr_bot_clear\6ca54494-76ff-11ee-8f2f-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\6f15d2de-76ff-11ee-94ec-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\4ae49614-76ff-11ee-b181-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\53e7f721-76ff-11ee-b51e-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\72623174-76ff-11ee-9e59-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\763bd8fe-76ff-11ee-a307-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\8ec9d7ba-76ff-11ee-ae25-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\5afa30bf-76ff-11ee-b5af-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\55278a59-76ff-11ee-8965-c09bf4619c03.mp3
Checking audio file path: ../data/train/hr_bot_clear\3bc9a05a-76ff-11ee-8f44-c09bf4619c03.mp3
Checking audio file 

Шаг 2: Обработка аудио и извлечение признаков
Для классификации команд нам нужно извлечь признаки из аудиофайлов. Мы можем использовать MFCC (Mel-Frequency Cepstral Coefficients) как набор признаков.

In [42]:
# Извлечение признаков
def extract_features(audio_filepath):
    try:
        # Загрузка аудиофайла
        signal, sr = librosa.load(audio_filepath, sr=None)
        # Извлечение MFCC
        mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print(f"Ошибка при обработке {audio_filepath}: {e}")
        return None

# Извлечение признаков
df['features'] = df['audio_filepath'].apply(extract_features)
df = df.dropna()  # Удаляем строки с ошибками

Шаг 3: Подготовка данных для обучения
Теперь мы разделим данные на обучающую и тестовую выборки и подготовим их для нейросети.

In [43]:
# Подготовка входных данных и меток
X = np.array(df['features'].tolist())
y = np.array(df['label'])

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Шаг 4: Создание нейронной сети
Мы можем использовать библиотеку TensorFlow или PyTorch для создания нейронной сети. Здесь мы создадим простую модель с использованием Keras.

In [44]:
# Создание модели
model = keras.Sequential([
    layers.Input(shape=(13,)),  # Входной слой
    layers.Dense(64, activation='relu'),  # Первый скрытый слой
    layers.Dense(64, activation='relu'),  # Второй скрытый слой
    layers.Dense(23, activation='softmax')  # Выходной слой для 23 классов
])

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Шаг 5: Обучение модели
Теперь мы можем обучить модель на подготовленных данных.

In [45]:
# Обучение модели
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2517 - loss: 21.7772 - val_accuracy: 0.4050 - val_loss: 2.8892
Epoch 2/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3952 - loss: 2.5283 - val_accuracy: 0.4076 - val_loss: 2.4787
Epoch 3/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4005 - loss: 2.3680 - val_accuracy: 0.4471 - val_loss: 2.2803
Epoch 4/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4163 - loss: 2.2026 - val_accuracy: 0.4523 - val_loss: 2.2107
Epoch 5/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4218 - loss: 2.2065 - val_accuracy: 0.4600 - val_loss: 2.2650
Epoch 6/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4315 - loss: 2.1727 - val_accuracy: 0.2760 - val_loss: 2.4848
Epoch 7/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4160 - loss: 2.2208 - val_accuracy: 0.4557 - val_loss: 2.2037
Epoch 8/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4387 - loss: 2.0883 - val_accuracy: 0

Шаг 6: Тестирование модели на валидационном наборе
После обучения мы можем протестировать модель на валидационном наборе.

In [46]:
# Оценка модели на валидационном наборе
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.2122 - loss: 2.7640
Validation Loss: 2.7242181301116943, Validation Accuracy: 0.21926052868366241


Шаг 7: Сохранение модели
После завершения обучения мы можем сохранить модель на диск для дальнейшего использования.

In [47]:
# Сохранение модели
model.save('train_command_model.h5')

Шаг 8: Использование модели
Теперь мы можем загрузить модель и использовать её для предсказания на новых аудиофайлах.

In [48]:
# Загрузка модели
loaded_model = keras.models.load_model('train_command_model.h5')

# Функция для предсказания
def predict_command(audio_filepath):
    features = extract_features(audio_filepath)
    if features is not None:
        features = np.reshape(features, (1, -1))  # Изменяем размерность
        prediction = model.predict(features)
        predicted_label = np.argmax(prediction)
        return predicted_label
    return None

# Пример предсказания
predicted_label = predict_command('../data/train/hr_bot_clear/3a0cb44f-76ff-11ee-844c-c09bf4619c03.mp3')  # Укажите путь к новому аудиофайлу
print(f"Предсказанная команда: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Предсказанная команда: 10


In [56]:
import os
import json
import numpy as np

# Константы
VAL_DIR = '../data/val/luga/'  # Путь к валидационным данным
ANNOTATION_FILE = os.path.join(VAL_DIR, 'luga.json')

# Загрузка аннотаций
def load_annotations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Загрузка аудиофайла и получение предсказания
def predict(audio_path):
    # Здесь должна быть логика вашей нейросети
    # Например, вы можете использовать вашу модель для предсказания:
    # prediction = model.predict(audio_path)

    # Для примера, будем возвращать случайное число как предсказание
    return np.random.randint(0, 30)  # Предположим, что метки от 0 до 29

# Основная функция для проверки команд
def evaluate_commands():
    annotations = load_annotations(ANNOTATION_FILE)

    correct_predictions = 0
    total_predictions = len(annotations)

    # Словарь для хранения статистики
    statistics = {
        'correct': {},
        'total': {},
        'accuracy': {}
    }

    # Обработка каждой аннотации
    for item in annotations:
        audio_filepath = item['audio_filepath']
        label = item['label']

        # Формирование полного пути к аудиофайлу
        audio_path = os.path.join(VAL_DIR, os.path.dirname(audio_filepath), os.path.basename(audio_filepath))

        if os.path.exists(audio_path):
            prediction = predict(audio_path)  # Получаем предсказание
            print(f'Predicted: {prediction}, Actual: {label} for {audio_path}')

            # Сохраняем статистику
            if label not in statistics['correct']:
                statistics['correct'][label] = 0
                statistics['total'][label] = 0

            statistics['total'][label] += 1

            if prediction == label:
                correct_predictions += 1
                statistics['correct'][label] += 1
        else:
            print(f'Audio file does not exist: {audio_path}')

    # Вычисляем общую точность
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

    # Сохраняем точность в статистике
    statistics['accuracy'] = accuracy

    # Выводим статистику
    print(f'\nTotal Predictions: {total_predictions}')
    print(f'Correct Predictions: {correct_predictions}')
    print(f'Accuracy: {accuracy:.2f}')
    print('Detailed statistics per label:')
    for label in statistics['total']:
        total = statistics['total'][label]
        correct = statistics['correct'].get(label, 0)
        print(f'Label: {label}, Total: {total}, Correct: {correct}, Accuracy: {correct / total:.2f}' if total > 0 else f'Label: {label}, Total: {total}, Correct: {correct}')

# Запуск проверки команд
evaluate_commands()


Predicted: 4, Actual: 17 for ../data/val/luga/21_11_2023\2023_11_21__09_54_58.wav
Predicted: 17, Actual: 20 for ../data/val/luga/15_11_2023\2023_11_15__11_38_51.wav
Predicted: 18, Actual: 4 for ../data/val/luga/02_11_2023\2023_11_02__10_41_09.wav
Predicted: 13, Actual: 6 for ../data/val/luga/15_11_2023\2023_11_15__11_38_41.wav
Predicted: 16, Actual: 7 for ../data/val/luga/15_11_2023\2023_11_15__09_41_25.wav
Predicted: 24, Actual: 10 for ../data/val/luga/15_11_2023\2023_11_15__10_31_56.wav
Predicted: 22, Actual: 7 for ../data/val/luga/03_07_2023\3fae175c-846e-11ee-a0ce-c09bf4619c03.wav
Predicted: 26, Actual: 15 for ../data/val/luga/21_11_2023\2023_11_21__10_48_45.wav
Predicted: 17, Actual: 4 for ../data/val/luga/03_07_2023\3f995bb0-846e-11ee-917f-c09bf4619c03.wav
Predicted: 6, Actual: 4 for ../data/val/luga/03_07_2023\3f9e302f-846e-11ee-9066-c09bf4619c03.wav
Predicted: 8, Actual: 4 for ../data/val/luga/21_11_2023\2023_11_21__10_10_55.wav
Predicted: 17, Actual: 19 for ../data/val/luga/03